# Check the implementation from:
https://geemap.org/notebooks/31_unsupervised_classification/ 

In [1]:
import geemap, ee

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

<IPython.core.display.Javascript object>

## Change to Sentinel 2 TOA and check cloud covers; change ROI and clustering ROI for better accuracy!

In [3]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2019-01-01', '2019-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

In [4]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.03,
 'CLOUD_COVER_LAND': 0.04,
 'EARTH_SUN_DISTANCE': 1.016591,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 6.348,
 'GEOMETRIC_RMSE_MODEL_X': 4.429,
 'GEOMETRIC_RMSE_MODEL_Y': 4.547,
 'IMAGE_DATE': '2019-07-12',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_022031_20190712_20190719_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1563565308000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-07-12T16:28:51.3794760Z',
 'SOLAR_AZIMUTH_ANGLE': 131.949371,
 'SOLAR_ZENITH_ANGLE': 26.494972,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 22,
 'WRS_ROW': 31,
 'system:asset_size': '553.046839 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_022031_20190712',
 'system:index': 'LC08_022031_20190712',
 'system:time_end': '2019-07-12 16:28:51',
 'system:time_start': '2019-07-12 16:28:51',
 'system:version': 1564390084

In [5]:
props.get('IMAGE_DATE').getInfo()

'2019-07-12'

In [6]:
props.get('CLOUD_COVER').getInfo()

0.03

# Construct Training Data Set:

In [7]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [8]:
# Make the training dataset.
training = image.sample(
    **{
        # 'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=24657.0, center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [9]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [10]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=24657.0, center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [11]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(bottom=24657.0, center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [12]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [13]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [14]:
geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\chany\Downloads\cluster.tif


In [15]:
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)